# Nombre: Andrés Degollado Muñoz
# Asignatura: Seminario de Investigación de Operaciones
# Proyecto final: Minimización de ciclos hamiltonianos en gráficas bipartitas completas

# Funciones elementales

In [ ]:
# Se importan las bibliotecas a usar.
import itertools
import numpy as np
import random
import math
import time

# Representa la lista de ciclos hamiltonianos de la gráfica K_{3,3}.
lista_soluciones= [[0,1,2,3,4,5], [0,1,2,5,4,3], [0,3,2,1,4,5], [0,3,2,5,4,1], [0,5,2,1,4,3], [0,5,2,3,4,1]]

# Esta función permite obtener los vecinos de una solución.
def vecindad_proyecto(lista):
    # Se crea una lista vacía para almacenar los vecinos.
    vecindad = []
    # Se eligen todos los vértices de la solución.
    vecino = lista[:]
    # Se extraen los elementos de las posiciones pares de la lista original.
    pos_par = vecino[1::2]
    # Se obtienen todas las permutaciones de los elementos de las posiciones pares.
    perm_hechas = list(itertools.permutations(pos_par))
    # Se iteran sobre las permutaciones obtenidas para devolver los vértices permutados.
    for perm in perm_hechas:
        # Se crea una copia del vecino actual.
        nuevo_vecino = vecino[:]
        # Se reemplazan los elementos pares por las permutaciones obtenidas.
        nuevo_vecino[1::2] = perm
        # Agrega el nuevo vecino a la vecindad.
        vecindad.append(nuevo_vecino)
    # Se elimina la solución original de la lista de vecinos.
    vecindad.pop(0)
    # Devuelve la vecindad generada.
    return vecindad

# Esta función genera la matriz de distancias para una gráfica bipartita,
# teniendo en cuenta un rango limitado de coordenadas.
def distancias_grafica_bipartita(tam_part_1, tam_part_2, rango=(0, 10)):
    # Se generan aleatoriamente las coordenadas de los nodos en un espacio bidimensional,
    # para cada subconjunto de la gráfica bipartita.
    part_1_coord = np.random.uniform(rango[0], rango[1], (tam_part_1, 2))
    part_2_coord = np.random.uniform(rango[0], rango[1], (tam_part_2, 2))

    # Se determina el número total de vértices.
    num_vertices = tam_part_1 + tam_part_2

    # Se construye la matriz de distancias.
    mat_dist = np.zeros((num_vertices, num_vertices))
    for i in range(tam_part_1):
        for j in range(tam_part_1, num_vertices):
            # Se aplica la norma euclidiana para determinar el peso de cada arista.
            distancia = np.linalg.norm(part_1_coord[i] - part_2_coord[j - tam_part_1])
            # Se agregan las distancias a la matriz de distancias.
            mat_dist[i, j] = distancia
            # Ya que la matriz de distancias es simétrica.
            mat_dist[j, i] = distancia

    return mat_dist

# Esta función evalúa la distancia total presente en un ciclo hamiltoniano.
def distancia_ciclo(g,tour):
        # Se suman las distancias entre cada par de vértices consecutivos,
        # salvo el penúltimo vértice con el último.
        dist = sum(g[tour[i]][tour[i+1]] for i in range(len(tour)-1))

        # Se suma el peso (distancia) entre el penúltimo vértice y el último.
        dist += g[tour[-1]][tour[0]]
        return dist

# Códigos de las metaheurísticas

In [ ]:
# Esta función representa la metaheurística de Búsqueda Localcon para encontrar el ciclo hamiltoniano de menor costo.
def Busqueda_local(solucion_inicial, grafica, funcion_objetivo, vecindad):
    """
    Parámetros:
    solucion_inicial (list): La solución inicial a partir de la cual se realiza el recocido simulado.
    grafica (object): La gráfica sobre la que se realiza el recocido simulado.
    funcion_objetivo (fun): La función objetivo que se quiere minimizar.
    vecindad (object): La vecindad que se va a trabajar en el proyecto.

    Retorna:
    mejor_solucion (list): La mejor solución encontrada.
    mejor_valor (float): La distancia total de la mejor solución.
    """
    # Se declara la solución inicial.
    s = solucion_inicial
    # Si la solución no es óptima de manera local, entonces encuentra una mejor solución.
    while not optimo_local(s, grafica, funcion_objetivo, vecindad):
        # Se hace la búsqueda de vecinos y se encuentra a un mejor vecino.
        s_prim = encontrar_mejor_vecino(s, grafica, funcion_objetivo, vecindad)
        # Declara al mejor vecino como la solución y lo devuelve.
        s = s_prim
    return s

# Esta función compara una solución con sus vecinos y determina si es mejor o no.
def optimo_local(s, grafica, funcion_objetivo, vecindad):
    for s_prim in vecindad(s):
        # Si la solución actual es peor que la del vecino, entonces regresa falso para que encuentre otro vecino.
        if funcion_objetivo(grafica, s) > funcion_objetivo(grafica, s_prim):
            return False
    return True

# Esta función selecciona a los mejores vecinos y selecciona uno aleatoriamente.
def encontrar_mejor_vecino(s, grafica, funcion_objetivo, vecindad):
    # Se raliza la filtración de los vecinos.
    mejores_vecinos = [s_prim for s_prim in vecindad(s) if funcion_objetivo(grafica, s) > funcion_objetivo(grafica, s_prim)]
    if mejores_vecinos:
        # Se selecciona un vecino al azar, en caso de que haya encontrado.
        return random.choice(mejores_vecinos)
    else:
        # De lo contrario, esto quiere decir que no se encontraron mejores vecinos.
        return s

In [ ]:
# Esta función construye una solución utilizando la construcción V.
def construccion_v(tours, grafica, funcion_objetivo, alpha):
    # Se calcula el valor objetivo de todos los tours.
    valores_tours = [(tour, funcion_objetivo(grafica, tour)) for tour in tours]

    # Se calculan las distancias mínima y máxima.
    distancias = [valor for _, valor in valores_tours]
    min_distancia = min(distancias)
    max_distancia = max(distancias)

    # Se crea la cota superior de la construcción V.
    cota = min_distancia + alpha * (max_distancia - min_distancia)
    # Crea la lista restringida de candidatos.
    lrc = [tour for tour, valor in valores_tours if valor <= cota]

    # Selecciona aleatoriamente un tour de la lista restringida de candidatos.
    tour_seleccionado = random.choice(lrc)

    return tour_seleccionado

# Esta función representa el método de GRASP con la construcción V para encontrar el ciclo hamiltoniano de menor costo.
def grasp_v(grafica, tours, funcion_objetivo, vecindad, max_iter, alpha):
    """
    Parámetros:
    grafica (object): La gráfica sobre la que se realiza la búsqueda.
    tours (object): El espacio de búsqueda (todos los ciclos hamiltonianos que hay en la gráfica).
    funcion_objetivo (función): La función objetivo que se quiere minimizar.
    vecindad (función): La función que genera los vecinos de una solución.
    max_iter (int): Número máximo de iteraciones (default: 100).
    alpha (float): Valor de filtración de la lista restringida de candidatos.

    Retorna:
    mejor_solucion (list): La mejor solución encontrada.
    mejor_valor (float): La distancia total de la mejor solución.
    """
    mejor_solucion = None
    mejor_valor = float('inf')

    for _ in range(max_iter):
        # Se utiliza la construcción V para obtener la solución inicial.
        solucion_actual = construccion_v(tours, grafica, funcion_objetivo, alpha)
        # Se aplica la Búsqueda Local a la solución inicial.
        #solucion_actual = Busqueda_local(solucion_inicial, grafica, funcion_objetivo, vecindad)
        # Se evalúa si la solución actual es mejor que la mejor solución encontrada hasta ahora.
        valor_actual = funcion_objetivo(grafica, solucion_actual)
        if valor_actual < mejor_valor:
            # Se actualiza la mejor solución.
            mejor_solucion = solucion_actual
            # Se actualiza el mejor valor.
            mejor_valor = valor_actual

    return mejor_solucion, mejor_valor

In [ ]:
# Esta función representa el método de recocido simulado para encontrar el ciclo hamiltoniano de menor costo.
def Recocido_simulado(solucion_inicial, grafica, funcion_objetivo, t0, alpha, NREP, tf):
    """
    Parámetros:
    solucion_inicial (list): La solución inicial a partir de la cual se realiza el recocido simulado.
    grafica (object): La gráfica sobre la que se realiza el recocido simulado.
    funcion_objetivo (fun): La función objetivo que se quiere minimizar.
    t0 (float): La temperatura inicial del sistema.
    alpha(float): El factor de enfriamiento que controla cómo disminuye la temperatura en cada iteración.
    NREP (int): El número de iteraciones a realizar a cada temperatura.
    tf (float): La temperatura final del sistema.

    Retorna:
    mejor_solucion (list): La mejor solución encontrada.
    mejor_valor (float): La distancia total de la mejor solución.
    """

    # Se iguala la mejor solución con la solución inicial.
    mejor_solucion = solucion_inicial
    # Se evalúa dicha solución en la función objetivo.
    mejor_valor = funcion_objetivo(grafica, mejor_solucion)
    # Se iguala la temperatura inicial con la actual.
    temperatura_actual = t0

    # Mientras la temperatura actual sea mayor a la temperatura final.
    while temperatura_actual > tf:
        # Se aplica el criterio de paro (número de iteraciones).
        for _ in range(NREP):
            # Se escoge aleatoriamente un vecino de la solución.
            vecino = random.choice(vecindad_proyecto(mejor_solucion))
            # Se obtiene la distancia total de la solución vecina.
            valor_vecino = funcion_objetivo(grafica, vecino)
            # Se aplica la diferencia entre el valor del vecino y el valor de la "mejor solución".
            diferencia = valor_vecino - mejor_valor
            # Si la diferencia es negativa, se actualiza el vecino como mejor solución y el mejor valor.
            if diferencia < 0:
                mejor_solucion = vecino
                mejor_valor = valor_vecino
            # Si el número aleatorio generado entre cero y uno es menor al factor de Boltzmann,
            # se actualiza el vecino como mejor solución y el mejor valor.
            elif random.random() < math.exp(-diferencia / temperatura_actual):
                mejor_solucion = vecino
                mejor_valor = valor_vecino

        # Se actualiza la temperatura actual.
        temperatura_actual *= alpha

    return mejor_solucion, mejor_valor

In [ ]:
# Esta función representa el método de búsqueda tabú para encontrar el ciclo hamiltoniano de menor costo.
def Busqueda_tabu(solucion_inicial, grafica, funcion_objetivo, lista_tabu, max_iter, iter_proh):
    """
    Implementa la búsqueda tabú para encontrar una mejor solución.

    Parámetros:
    solucion_inicial (list): La solución inicial a partir de la cual se realiza la búsqueda tabú.
    grafica (object): La gráfica sobre la que se realiza la búsqueda.
    funcion_objetivo (fun): La función objetivo que se quiere minimizar.
    lista_tabu (list): La lista tabú que contiene las soluciones prohibidas.
    max_iter (int): Número máximo de iteraciones.
    iter_proh (int): Número de iteraciones prohibidas para una solución perteneciente a la lista tabú.

    Retorna:
    mejor_solucion (list): La mejor solución encontrada.
    mejor_valor (float): La distancia total de la mejor solución.
    """

    # Se iguala la mejor solución con la solución inicial
    mejor_solucion = solucion_inicial
    # Se evalúa dicha solución en la función objetivo.
    mejor_valor = funcion_objetivo(grafica, mejor_solucion)
    # Se crea un índice auxiliar para las iteraciones.
    iteracion_actual = 0
    # Mientras la iteración actual sea menor al máximo número de iteraciones.
    while iteracion_actual < max_iter:
        # Se genera la vecindad de la solución actual.
        vecindario = vecindad_proyecto(mejor_solucion)
        # Se evalúan todas las soluciones en el vecindario.
        for vecino in vecindario:
            # Si el vecino no está en la lista tabú, se obtiene su distancia total.
            if vecino not in lista_tabu:
                valor_vecino = funcion_objetivo(grafica, vecino)
                # Si el valor del vecino es mejor que el valor actual, se actualiza
                # la mejor solución y su mejor valor.
                if valor_vecino < mejor_valor:
                    mejor_solucion = vecino
                    mejor_valor = valor_vecino

        # Se actualiza la lista tabú.
        lista_tabu.append(mejor_solucion)
        # Se aplica la prohibición por la cantidad iteraciones definidas por el usuario.
        if len(lista_tabu) > iter_proh:
            lista_tabu.pop(0)

        # Se actualiza el iterador.
        iteracion_actual += 1

    return mejor_solucion, mejor_valor

In [ ]:
# Esta función crea una población inicial a partir de tours totales.
def crear_poblacion(tours_totales, tam_poblacion):
    # Se crea la población inicial seleccionando aleatoriamente cierta cantidad de tours.
    poblacion = random.sample(tours_totales, tam_poblacion)
    return poblacion

# Esta función representa la selección por ruleta
def seleccion_ruleta(poblacion, fitness):
    # Se calcula la suma total de los valores de aptitud de toda la población.
    max_fit = sum(fitness)
    # Se genera un número aleatorio entre 0 y la suma total de aptitudes.
    selecto = random.uniform(0, max_fit)
    # Se inicializa una variable acumulativa para recorrer los valores de aptitud.
    actual = 0
    for i, individuo in enumerate(poblacion):
        # Se itera sobre la población, sumando los valores de aptitud acumulativamente.
        actual += fitness[i]
        # Si el valor acumulado supera el número aleatorio generado,
        # se selecciona el individuo actual.
        if actual > selecto:
            return individuo

# Esta función representa el cruce que sucede
# al intercambiar vértices en posiciones pares
def cruce(padre1, padre2):
    # Calcula la longitud del primer padre.
    n = len(padre1)
    # Crea una lista llamada hijo de longitud n y la inicializa con -1 en todas las posiciones.
    # Esta lista representará al hijo resultante del cruce y -1 indica una posición no asignada.
    hijo = [-1] * n
    # Itera sobre los índices pares del primer padre e iguala los valores del padre al hijo.
    for i in range(0, n, 2):
        hijo[i] = padre1[i]
    # Itera sobre todos los índices del segundo padre.
    for i in range(n):
        # Comprueba si el valor del segundo en la posición i no está ya presente en el hijo.
        if padre2[i] not in hijo:
            # De cumplirse la condición anterior, se inicia un ciclo interno que itera sobre los índices del hijo.
            for j in range(n):
                # Comprueba si la posición j en el hijo está todavía asignada a -1.
                if hijo[j] == -1:
                    # Si encuentra una posición -1, se asigna el valor del segundo padre en la posición i a esta posición j del hijo
                    # y rompe el ciclo interno.
                    hijo[j] = padre2[i]
                    break
    return hijo

# Esta función representa la mutación que consiste en el intercambio solo en posiciones pares.
def mutacion(individuo, tasa_mutacion):
    # Itera solamente en las posiciones pares.
    for i in range(0, len(individuo), 2):
        # Si el número generado entre 0 y 1 es menor que la tasa de mutación
        if random.random() < tasa_mutacion:
            # Se hace un intercambio en las posiciones pares del individuo.
            j = random.choice(range(0, len(individuo), 2))
            individuo[i], individuo[j] = individuo[j], individuo[i]

# Esta función representa el algoritmo genético para encontrar el ciclo hamiltoniano de menor costo.
def algoritmo_genetico(g, tours_totales, tam_poblacion, num_generaciones, tasa_mutacion):
    # Se escoge una solución inicial aleatoria.
    solucion_inicial = random.choice(tours_totales)
    # Se crea la población.
    poblacion = crear_poblacion(tours_totales, tam_poblacion)
    for _ in range(num_generaciones):
        # Calcula el fitness de cada individuo en la población.
        # El fitness es inversamente proporcional a la distancia del ciclo,
        # con el fin de maximizar el número de individuos con la menor distancia.
        fitness = [1 / distancia_ciclo(g, ind) for ind in poblacion]
        # Se crea una lista vacía que representa a la nueva población.
        nueva_poblacion = []
        # El siguiente ciclo itera la mitad de las veces,
        # ya que cada iteración del bucle generará dos nuevos hijos de la nueva población.
        for _ in range(tam_poblacion // 2):
            # Se aplica la selección por ruleta a ambos padres.
            padre1 = seleccion_ruleta(poblacion, fitness)
            padre2 = seleccion_ruleta(poblacion, fitness)
            # Se aplica el cruce a los hijos.
            hijo1 = cruce(padre1, padre2)
            hijo2 = cruce(padre2, padre1)
            # Se aplica la tasa de mutación a los hijos.
            mutacion(hijo1, tasa_mutacion)
            mutacion(hijo2, tasa_mutacion)
            # Se añaden los dos nuevos individuos (hijos) a la lista nueva_población.
            nueva_poblacion.extend([hijo1, hijo2])
        # Se sustituye la población actual con la nueva población generada.
        poblacion = nueva_poblacion
    # Encuentra el mejor individuo en la población final.
    # El mejor individuo es el que tiene la menor distancia del ciclo,
    # usando la función objetivo como criterio.
    mejor_individuo = min(poblacion, key=lambda ind: distancia_ciclo(g, ind))
    # Encuentra el índice cero del mejor individuo.
    indice_cero = mejor_individuo.index(0)
    # Reorganiza la solución para que comience desde cero.
    mejor_individuo_ordenado = mejor_individuo[indice_cero:] + mejor_individuo[:indice_cero]
    return mejor_individuo_ordenado, distancia_ciclo(g, mejor_individuo)

# Sección de pruebas de las metaheurísticas para la gráfica bipartita completa $K_{3,3}$

## Código de la gráfica bipartita completa $K_{3,3}$

In [ ]:
# A continuación, se crea la matriz de distancias para la gráfica K_{3,3}.
tam_part_1 = 3
tam_part_2= 3
grafica_bipartita_k_3 = distancias_grafica_bipartita(tam_part_1,tam_part_2)

# Se define la lista de todos los posibles tours distintos en la gráfica $K_{3,3}$, con base en la matriz de distancias.
tours_totales_3=vecindad_proyecto([0,3,1,4,2,5])
tours_totales_3.append([0,3,1,4,2,5])

## Búsqueda Local y GRASP para la gráfica bipartita completa $K_{3,3}$

In [ ]:
# Búsqueda Local aplicada al problema.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_busqueda_local_k_3=time.time()
# Se proporciona una solución inicial al azar.
solucion_inicial_k_3 = random.choice(tours_totales_3)
# Se obtiene una mejor solución, usando Búsqueda Local para cada una de las instancias.
mejor_solucion_k_3 = Busqueda_local(solucion_inicial_k_3, grafica_bipartita_k_3, distancia_ciclo, vecindad_proyecto)
# Se imprime la mejor solución y su distancia total.
print("El ciclo hamiltoniano más corto con Búsqueda Local es",mejor_solucion_k_3,"cuya distancia total es",distancia_ciclo(grafica_bipartita_k_3, mejor_solucion_k_3),"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Local fue de %s segundos. " % (time.time() - t_inicial_busqueda_local_k_3),"\n")

# GRASP con construcción V aplicado al problema.

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.3.
t_inicial_grasp_v_k_3=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_3 = grasp_v(grafica_bipartita_k_3, tours_totales_3, distancia_ciclo, vecindad_proyecto, 10, 0.3)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es",mejor_solucion_grasp_v_k_3[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_3),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.5.
t_inicial_grasp_v_k_3_2=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_3_2 = grasp_v(grafica_bipartita_k_3, tours_totales_3, distancia_ciclo, vecindad_proyecto, 10, 0.5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es",mejor_solucion_grasp_v_k_3_2[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_3_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_3_2),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.7.
t_inicial_grasp_v_k_3_3=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_3_3 = grasp_v(grafica_bipartita_k_3, tours_totales_3, distancia_ciclo, vecindad_proyecto, 10, 0.7)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es",mejor_solucion_grasp_v_k_3_3[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_3_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_3_3),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.9.
t_inicial_grasp_v_k_3_4=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_3_4 = grasp_v(grafica_bipartita_k_3, tours_totales_3, distancia_ciclo, vecindad_proyecto, 10, 0.9)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.9 es",mejor_solucion_grasp_v_k_3_4[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_3_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_3_4),"\n")

El ciclo hamiltoniano más corto con Búsqueda Local es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo de Búsqueda Local fue de 0.004396677017211914 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo de GRASP con construcción V fue de 0.0017364025115966797 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo de GRASP con construcción V fue de 0.0006041526794433594 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo de GRASP con construcción V fue de 0.00048661231994628906 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.9 es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.314199

## Recocido Simulado para la gráfica bipartita completa $K_{3,3}$

In [ ]:
# Recocido Simulado aplicado al problema.

# Se establecen las temperaturas inicial y final
t0= 100
tf= 0.1
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_3=time.time()
# Se aplica el algoritmo al problema para alpha=0.6 y NREP=1.
mejor_solucion_recocido_k_3 = Recocido_simulado(solucion_inicial_k_3, grafica_bipartita_k_3, distancia_ciclo, t0, 0.6, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es",mejor_solucion_recocido_k_3[0],
      "cuya distancia total es",mejor_solucion_recocido_k_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_3),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_3_2=time.time()
# Se aplica el algoritmo al problema para alpha=0.7 y NREP=1.
mejor_solucion_recocido_k_3_2 = Recocido_simulado(solucion_inicial_k_3, grafica_bipartita_k_3, distancia_ciclo, t0, 0.7, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es",mejor_solucion_recocido_k_3_2[0],
      "cuya distancia total es",mejor_solucion_recocido_k_3_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_3_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_3_3=time.time()
# Se aplica el algoritmo al problema para alpha=0.8 y NREP=1.
mejor_solucion_recocido_k_3_3 = Recocido_simulado(solucion_inicial_k_3, grafica_bipartita_k_3, distancia_ciclo, t0, 0.8, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es",mejor_solucion_recocido_k_3_3[0],
      "cuya distancia total es",mejor_solucion_recocido_k_3_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_3_3),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_3_4=time.time()
# Se aplica el algoritmo al problema para alpha=0.9 y NREP=1.
mejor_solucion_recocido_k_3_4 = Recocido_simulado(solucion_inicial_k_3, grafica_bipartita_k_3, distancia_ciclo, t0, 0.9, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es",mejor_solucion_recocido_k_3_4[0],
      "cuya distancia total es",mejor_solucion_recocido_k_3_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_3_4),"\n")

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo de Recocido Simulado fue de 0.0013747215270996094 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es [0, 5, 1, 4, 2, 3] cuya distancia total es 30.096752058184105 unidades.
El tiempo de Recocido Simulado fue de 0.0006480216979980469 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo de Recocido Simulado fue de 0.0011639595031738281 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo de Recocido Simulado fue de 0.0013282299041748047 segundos.  



## Búsqueda Tabú para la gráfica bipartita completa $K_{3,3}$

In [ ]:
# Búsqueda Tabú aplicada al problema.

# Se eligen aleatoriamente una cantidad de tours a prohibir.
tours_prohibidos_k_3=random.sample(tours_totales_3, 2)
# Se crea la lista tabú.
lista_tabu_k_3 = []
# Se agregan los ciclos hamiltonianos prohibidos a la lista.
lista_tabu_k_3.append(tours_prohibidos_k_3)
# Se elige aleatoriamente una solución inicial.
solucion_inicial_tabu_k_3 = random.choice(tours_totales_3)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_3=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_3 = Busqueda_tabu(solucion_inicial_tabu_k_3, grafica_bipartita_k_3, distancia_ciclo, lista_tabu_k_3, 10, 1)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_3[0],
      "cuya distancia total es",mejor_solucion_tabu_k_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_3),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_3_2=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_3_2 = Busqueda_tabu(solucion_inicial_tabu_k_3, grafica_bipartita_k_3, distancia_ciclo, lista_tabu_k_3, 10, 5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_3_2[0],
      "cuya distancia total es",mejor_solucion_tabu_k_3_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_3_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_3_3=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_3_3 = Busqueda_tabu(solucion_inicial_tabu_k_3, grafica_bipartita_k_3, distancia_ciclo, lista_tabu_k_3, 10, 10)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_3_3[0],
      "cuya distancia total es",mejor_solucion_tabu_k_3_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_3_3),"\n")

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo de Búsqueda Tabú fue de 0.0006074905395507812 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo de Búsqueda Tabú fue de 0.0004558563232421875 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 5, 1, 3, 2, 4] cuya distancia total es 29.60736638469375 unidades.
El tiempo de Búsqueda Tabú fue de 0.0010445117950439453 segundos  



## Algoritmos Genéticos para la gráfica bipartita completa $K_{3,3}$

In [ ]:
# Algoritmo Genético aplicado al problema.


# Se asigna el tamaño de la población que sea menor o igual a la longitud de tours_totales
tam_poblacion_k_3 = len(tours_totales_3)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_3=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 15%.
mejor_solucion_geneticos_k_3 = algoritmo_genetico(grafica_bipartita_k_3, tours_totales_3, tam_poblacion_k_3, 20, 0.15)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es", mejor_solucion_geneticos_k_3[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_3),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_3_2=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 20%.
mejor_solucion_geneticos_k_3_2 = algoritmo_genetico(grafica_bipartita_k_3, tours_totales_3, tam_poblacion_k_3, 20, 0.2)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es", mejor_solucion_geneticos_k_3_2[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_3_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_3_2),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_3_3=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 25%.
mejor_solucion_geneticos_k_3_3 = algoritmo_genetico(grafica_bipartita_k_3, tours_totales_3, tam_poblacion_k_3, 20, 0.25)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es", mejor_solucion_geneticos_k_3_3[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_3_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_3_3),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es [0, 3, 2, 5, 1, 4] cuya distancia total es 29.31419975276194 unidades.
El tiempo del Algoritmo Genético fue de 0.001611471176147461 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es [0, 4, 2, 3, 1, 5] cuya distancia total es 29.60736638469375 unidades.
El tiempo del Algoritmo Genético fue de 0.0016243457794189453 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es [0, 4, 1, 5, 2, 3] cuya distancia total es 29.31419975276194 unidades.
El tiempo del Algoritmo Genético fue de 0.0016622543334960938 segundos  



# Sección de pruebas de las metaheurísticas para la gráfica bipartita completa $K_{4,4}$

## Código de la gráfica bipartita completa $K_{4,4}$

In [ ]:
# A continuación, se crea la matriz de distancias para la gráfica K_{4,4}.
tam_part_3 = 4
tam_part_4= 4
grafica_bipartita_k_4 = distancias_grafica_bipartita(tam_part_3,tam_part_4)

# Se define la lista de todos los posibles tours distintos en la gráfica $K_{4,4}$, con base en la matriz de distancias.
tours_totales_4=vecindad_proyecto([0,4,1,5,2,6,3,7])
tours_totales_4.append([0,4,1,5,2,6,3,7])

## Búsqueda Local y GRASP para la gráfica bipartita completa $K_{4,4}$

In [ ]:
# Búsqueda Local aplicada al problema.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_busqueda_local_k_4=time.time()
# Se proporciona una solución inicial al azar.
solucion_inicial_k_4 = random.choice(tours_totales_4)
# Se obtiene una mejor solución, usando Búsqueda Local para cada una de las instancias.
mejor_solucion_k_4 = Busqueda_local(solucion_inicial_k_4, grafica_bipartita_k_4, distancia_ciclo, vecindad_proyecto)
# Se imprime la mejor solución y su distancia total.
print("El ciclo hamiltoniano más corto con Búsqueda Local es",mejor_solucion_k_4,"cuya distancia total es",distancia_ciclo(grafica_bipartita_k_4, mejor_solucion_k_4),"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Local fue de %s segundos. " % (time.time() - t_inicial_busqueda_local_k_4),"\n")

# GRASP con construcción V aplicado al problema.

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.3.
t_inicial_grasp_v_k_4=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_4 = grasp_v(grafica_bipartita_k_4, tours_totales_4, distancia_ciclo, vecindad_proyecto, 10, 0.3)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es",mejor_solucion_grasp_v_k_4[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_4),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.5.
t_inicial_grasp_v_k_4_2=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_4_2 = grasp_v(grafica_bipartita_k_4, tours_totales_4, distancia_ciclo, vecindad_proyecto, 10, 0.5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es",mejor_solucion_grasp_v_k_4_2[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_4_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_4_2),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.7.
t_inicial_grasp_v_k_4_3=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_4_3 = grasp_v(grafica_bipartita_k_4, tours_totales_4, distancia_ciclo, vecindad_proyecto, 10, 0.7)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es",mejor_solucion_grasp_v_k_4_3[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_4_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_4_3),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.9.
t_inicial_grasp_v_k_4_4=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_4_4 = grasp_v(grafica_bipartita_k_4, tours_totales_4, distancia_ciclo, vecindad_proyecto, 10, 0.9)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.9 es",mejor_solucion_grasp_v_k_4_4[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_4_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_4_4),"\n")

El ciclo hamiltoniano más corto con Búsqueda Local es [0, 7, 1, 5, 2, 4, 3, 6] cuya distancia total es 36.84031223147192 unidades.
El tiempo de Búsqueda Local fue de 0.003844022750854492 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es [0, 7, 1, 5, 2, 4, 3, 6] cuya distancia total es 36.84031223147192 unidades.
El tiempo de GRASP con construcción V fue de 0.005327463150024414 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es [0, 5, 1, 7, 2, 4, 3, 6] cuya distancia total es 36.98184911392816 unidades.
El tiempo de GRASP con construcción V fue de 0.0018167495727539062 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es [0, 7, 1, 5, 2, 4, 3, 6] cuya distancia total es 36.84031223147192 unidades.
El tiempo de GRASP con construcción V fue de 0.002277374267578125 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.9 es [0, 7, 1, 5, 2, 4, 3, 6] cuya d

## Recocido Simulado para la gráfica bipartita completa $K_{4,4}$

In [ ]:
# Recocido Simulado aplicado al problema.

# Se establecen las temperaturas inicial y final
t0= 100
tf= 0.1
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_4=time.time()
# Se aplica el algoritmo al problema para alpha=0.6 y NREP=1.
mejor_solucion_recocido_k_4 = Recocido_simulado(solucion_inicial_k_4, grafica_bipartita_k_4, distancia_ciclo, t0, 0.6, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es",mejor_solucion_recocido_k_4[0],
      "cuya distancia total es",mejor_solucion_recocido_k_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_4),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_4_2=time.time()
# Se aplica el algoritmo al problema para alpha=0.7 y NREP=1.
mejor_solucion_recocido_k_4_2 = Recocido_simulado(solucion_inicial_k_4, grafica_bipartita_k_4, distancia_ciclo, t0, 0.7, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es",mejor_solucion_recocido_k_4_2[0],
      "cuya distancia total es",mejor_solucion_recocido_k_4_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_4_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_4_3=time.time()
# Se aplica el algoritmo al problema para alpha=0.8 y NREP=1.
mejor_solucion_recocido_k_4_3 = Recocido_simulado(solucion_inicial_k_4, grafica_bipartita_k_4, distancia_ciclo, t0, 0.8, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es",mejor_solucion_recocido_k_4_3[0],
      "cuya distancia total es",mejor_solucion_recocido_k_4_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_4_3),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_4_4=time.time()
# Se aplica el algoritmo al problema para alpha=0.9 y NREP=1.
mejor_solucion_recocido_k_4_4 = Recocido_simulado(solucion_inicial_k_4, grafica_bipartita_k_4, distancia_ciclo, t0, 0.9, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es",mejor_solucion_recocido_k_4_4[0],
      "cuya distancia total es",mejor_solucion_recocido_k_4_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_4_4),"\n")

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es [0, 5, 1, 7, 2, 6, 3, 4] cuya distancia total es 38.02296378935823 unidades.
El tiempo de Recocido Simulado fue de 0.0005977153778076172 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es [0, 7, 1, 5, 2, 4, 3, 6] cuya distancia total es 36.84031223147192 unidades.
El tiempo de Recocido Simulado fue de 0.0006432533264160156 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es [0, 5, 1, 7, 2, 4, 3, 6] cuya distancia total es 36.98184911392816 unidades.
El tiempo de Recocido Simulado fue de 0.001262664794921875 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es [0, 5, 1, 7, 2, 4, 3, 6] cuya distancia total es 36.98184911392816 unidades.
El tiempo de Recocido Simulado fue de 0.0027670860290527344 segundos.  



## Búsqueda Tabú para la gráfica bipartita completa $K_{4,4}$

In [ ]:
# Búsqueda Tabú aplicada al problema.

# Se eligen aleatoriamente una cantidad de tours a prohibir.
tours_prohibidos_k_4=random.sample(tours_totales_4, 6)
# Se crea la lista tabú.
lista_tabu_k_4 = []
# Se agregan los ciclos hamiltonianos prohibidos a la lista.
lista_tabu_k_4.append(tours_prohibidos_k_4)
# Se elige aleatoriamente una solución inicial.
solucion_inicial_tabu_k_4 = random.choice(tours_totales_4)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_4=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_4 = Busqueda_tabu(solucion_inicial_tabu_k_4, grafica_bipartita_k_4, distancia_ciclo, lista_tabu_k_4, 10, 1)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_4[0],
      "cuya distancia total es",mejor_solucion_tabu_k_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_4),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_4_2=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_4_2 = Busqueda_tabu(solucion_inicial_tabu_k_4, grafica_bipartita_k_4, distancia_ciclo, lista_tabu_k_4, 10, 5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_4_2[0],
      "cuya distancia total es",mejor_solucion_tabu_k_4_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_4_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_4_3=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_4_3 = Busqueda_tabu(solucion_inicial_tabu_k_4, grafica_bipartita_k_4, distancia_ciclo, lista_tabu_k_4, 10, 10)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_4_3[0],
      "cuya distancia total es",mejor_solucion_tabu_k_4_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_4_3),"\n")

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 7, 1, 5, 2, 4, 3, 6] cuya distancia total es 36.84031223147192 unidades.
El tiempo de Búsqueda Tabú fue de 0.001537322998046875 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 7, 1, 5, 2, 4, 3, 6] cuya distancia total es 36.84031223147192 unidades.
El tiempo de Búsqueda Tabú fue de 0.0013988018035888672 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 5, 1, 7, 2, 4, 3, 6] cuya distancia total es 36.98184911392816 unidades.
El tiempo de Búsqueda Tabú fue de 0.002084493637084961 segundos  



## Algoritmos Genéticos para la gráfica bipartita completa $K_{4,4}$

In [ ]:
# Algoritmo Genético aplicado al problema.


# Se asigna el tamaño de la población que sea menor o igual a la longitud de tours_totales
tam_poblacion_k_4 = len(tours_totales_4)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_4=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 15%.
mejor_solucion_geneticos_k_4 = algoritmo_genetico(grafica_bipartita_k_4, tours_totales_4, tam_poblacion_k_4, 20, 0.15)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es", mejor_solucion_geneticos_k_4[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_4),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_4_2=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 20%.
mejor_solucion_geneticos_k_4_2 = algoritmo_genetico(grafica_bipartita_k_4, tours_totales_4, tam_poblacion_k_4, 20, 0.2)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es", mejor_solucion_geneticos_k_4_2[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_4_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_4_2),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_4_3=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 25%.
mejor_solucion_geneticos_k_4_3 = algoritmo_genetico(grafica_bipartita_k_4, tours_totales_4, tam_poblacion_k_4, 20, 0.25)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es", mejor_solucion_geneticos_k_4_3[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_4_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_4_3),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es [0, 7, 2, 5, 1, 4, 3, 6] cuya distancia total es 37.530592096738594 unidades.
El tiempo del Algoritmo Genético fue de 0.009113311767578125 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es [0, 7, 2, 6, 3, 4, 1, 5] cuya distancia total es 39.16502433173986 unidades.
El tiempo del Algoritmo Genético fue de 0.016039609909057617 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es [0, 5, 1, 7, 2, 6, 3, 4] cuya distancia total es 38.02296378935823 unidades.
El tiempo del Algoritmo Genético fue de 0.011954069137573242 segundos  



# Sección de pruebas de las metaheurísticas para la gráfica bipartita completa $K_{5,5}$

## Código de la gráfica bipartita completa $K_{5,5}$

In [ ]:
# A continuación, se crea la matriz de distancias para la gráfica K_{5,5}.
tam_part_5 = 5
tam_part_6= 5
grafica_bipartita_k_5 = distancias_grafica_bipartita(tam_part_5,tam_part_6)

# Se define la lista de todos los posibles tours distintos en la gráfica $K_{5,5}$, con base en la matriz de distancias.
tours_totales_5=vecindad_proyecto([0,5,1,6,2,7,3,8,4,9])
tours_totales_5.append([0,5,1,6,2,7,3,8,4,9])

## Búsqueda Local y GRASP para la gráfica bipartita completa $K_{5,5}$

In [ ]:
# Búsqueda Local aplicada al problema.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_busqueda_local_k_5=time.time()
# Se proporciona una solución inicial al azar.
solucion_inicial_k_5 = random.choice(tours_totales_5)
# Se obtiene una mejor solución, usando Búsqueda Local para cada una de las instancias.
mejor_solucion_k_5 = Busqueda_local(solucion_inicial_k_5, grafica_bipartita_k_5, distancia_ciclo, vecindad_proyecto)
# Se imprime la mejor solución y su distancia total.
print("El ciclo hamiltoniano más corto con Búsqueda Local es",mejor_solucion_k_5,"cuya distancia total es",distancia_ciclo(grafica_bipartita_k_5, mejor_solucion_k_5),"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Local fue de %s segundos. " % (time.time() - t_inicial_busqueda_local_k_5),"\n")

# GRASP con construcción V aplicado al problema.

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.3.
t_inicial_grasp_v_k_5=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_5 = grasp_v(grafica_bipartita_k_5, tours_totales_5, distancia_ciclo, vecindad_proyecto, 10, 0.3)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es",mejor_solucion_grasp_v_k_5[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_5[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_5),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.5.
t_inicial_grasp_v_k_5_2=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_5_2 = grasp_v(grafica_bipartita_k_5, tours_totales_5, distancia_ciclo, vecindad_proyecto, 10, 0.5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es",mejor_solucion_grasp_v_k_5_2[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_5_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_5_2),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.7.
t_inicial_grasp_v_k_5_3=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_5_3 = grasp_v(grafica_bipartita_k_5, tours_totales_5, distancia_ciclo, vecindad_proyecto, 10, 0.7)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es",mejor_solucion_grasp_v_k_5_3[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_5_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_5_3),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.9.
t_inicial_grasp_v_k_5_4=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_5_4 = grasp_v(grafica_bipartita_k_5, tours_totales_5, distancia_ciclo, vecindad_proyecto, 10, 0.9)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.9 es",mejor_solucion_grasp_v_k_5_4[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_5_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_5_4),"\n")

El ciclo hamiltoniano más corto con Búsqueda Local es [0, 7, 1, 8, 2, 5, 3, 6, 4, 9] cuya distancia total es 44.898250022791295 unidades.
El tiempo de Búsqueda Local fue de 0.0197145938873291 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es [0, 7, 1, 6, 2, 5, 3, 8, 4, 9] cuya distancia total es 47.78841695046764 unidades.
El tiempo de GRASP con construcción V fue de 0.009797334671020508 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es [0, 5, 1, 8, 2, 7, 3, 6, 4, 9] cuya distancia total es 49.11857351967143 unidades.
El tiempo de GRASP con construcción V fue de 0.00914764404296875 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es [0, 7, 1, 9, 2, 5, 3, 8, 4, 6] cuya distancia total es 47.84559201801275 unidades.
El tiempo de GRASP con construcción V fue de 0.010171651840209961 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.9 es [0, 9, 1, 

## Recocido Simulado para la gráfica bipartita completa $K_{5,5}$

In [ ]:
# Recocido Simulado aplicado al problema.

# Se establecen las temperaturas inicial y final
t0= 100
tf= 0.1
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_5=time.time()
# Se aplica el algoritmo al problema para alpha=0.6 y NREP=1.
mejor_solucion_recocido_k_5 = Recocido_simulado(solucion_inicial_k_5, grafica_bipartita_k_5, distancia_ciclo, t0, 0.6, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es",mejor_solucion_recocido_k_5[0],
      "cuya distancia total es",mejor_solucion_recocido_k_5[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_5),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_5_2=time.time()
# Se aplica el algoritmo al problema para alpha=0.7 y NREP=1.
mejor_solucion_recocido_k_5_2 = Recocido_simulado(solucion_inicial_k_5, grafica_bipartita_k_5, distancia_ciclo, t0, 0.7, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es",mejor_solucion_recocido_k_5_2[0],
      "cuya distancia total es",mejor_solucion_recocido_k_5_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_5_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_5_3=time.time()
# Se aplica el algoritmo al problema para alpha=0.8 y NREP=1.
mejor_solucion_recocido_k_5_3 = Recocido_simulado(solucion_inicial_k_5, grafica_bipartita_k_5, distancia_ciclo, t0, 0.8, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es",mejor_solucion_recocido_k_5_3[0],
      "cuya distancia total es",mejor_solucion_recocido_k_5_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_5_3),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_5_4=time.time()
# Se aplica el algoritmo al problema para alpha=0.9 y NREP=1.
mejor_solucion_recocido_k_5_4 = Recocido_simulado(solucion_inicial_k_5, grafica_bipartita_k_5, distancia_ciclo, t0, 0.9, 1, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es",mejor_solucion_recocido_k_5_4[0],
      "cuya distancia total es",mejor_solucion_recocido_k_5_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_5_4),"\n")

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es [0, 7, 1, 9, 2, 5, 3, 8, 4, 6] cuya distancia total es 47.84559201801275 unidades.
El tiempo de Recocido Simulado fue de 0.006664752960205078 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es [0, 7, 1, 5, 2, 8, 3, 9, 4, 6] cuya distancia total es 49.4520802917475 unidades.
El tiempo de Recocido Simulado fue de 0.0045185089111328125 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es [0, 7, 1, 5, 2, 8, 3, 6, 4, 9] cuya distancia total es 46.97465104457325 unidades.
El tiempo de Recocido Simulado fue de 0.00944375991821289 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es [0, 7, 1, 8, 2, 5, 3, 6, 4, 9] cuya distancia total es 44.898250022791295 unidades.
El tiempo de Recocido Simulado fue de 0.009154081344604492 segundos.  



## Búsqueda Tabú para la gráfica bipartita completa $K_{5,5}$

In [ ]:
# Búsqueda Tabú aplicada al problema.

# Se eligen aleatoriamente una cantidad de tours a prohibir.
tours_prohibidos_k_5=random.sample(tours_totales_5, 20)
# Se crea la lista tabú.
lista_tabu_k_5 = []
# Se agregan los ciclos hamiltonianos prohibidos a la lista.
lista_tabu_k_5.append(tours_prohibidos_k_5)
# Se elige aleatoriamente una solución inicial.
solucion_inicial_tabu_k_5 = random.choice(tours_totales_5)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_5=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_5 = Busqueda_tabu(solucion_inicial_tabu_k_5, grafica_bipartita_k_5, distancia_ciclo, lista_tabu_k_5, 10, 1)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_5[0],
      "cuya distancia total es",mejor_solucion_tabu_k_5[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_5),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_5_2=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_5_2 = Busqueda_tabu(solucion_inicial_tabu_k_5, grafica_bipartita_k_5, distancia_ciclo, lista_tabu_k_5, 10, 5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_5_2[0],
      "cuya distancia total es",mejor_solucion_tabu_k_5_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_5_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_5_3=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_5_3 = Busqueda_tabu(solucion_inicial_tabu_k_5, grafica_bipartita_k_5, distancia_ciclo, lista_tabu_k_5, 10, 10)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_5_3[0],
      "cuya distancia total es",mejor_solucion_tabu_k_5_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_5_3),"\n")

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 7, 1, 8, 2, 5, 3, 6, 4, 9] cuya distancia total es 44.898250022791295 unidades.
El tiempo de Búsqueda Tabú fue de 0.007393836975097656 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 7, 1, 8, 2, 5, 3, 6, 4, 9] cuya distancia total es 44.898250022791295 unidades.
El tiempo de Búsqueda Tabú fue de 0.016451120376586914 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 7, 1, 5, 2, 8, 3, 6, 4, 9] cuya distancia total es 46.97465104457325 unidades.
El tiempo de Búsqueda Tabú fue de 0.010137796401977539 segundos  



## Algoritmos Genéticos para la gráfica bipartita completa $K_{5,5}$

In [ ]:
# Algoritmo Genético aplicado al problema.


# Se asigna el tamaño de la población que sea menor o igual a la longitud de tours_totales
tam_poblacion_k_5 = len(tours_totales_5)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_5=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 15%.
mejor_solucion_geneticos_k_5 = algoritmo_genetico(grafica_bipartita_k_5, tours_totales_5, tam_poblacion_k_5, 20, 0.15)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es", mejor_solucion_geneticos_k_5[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_5[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_5),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_5_2=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 20%.
mejor_solucion_geneticos_k_5_2 = algoritmo_genetico(grafica_bipartita_k_5, tours_totales_5, tam_poblacion_k_5, 20, 0.2)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es", mejor_solucion_geneticos_k_5_2[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_5_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_5_2),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_5_3=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 25%.
mejor_solucion_geneticos_k_5_3 = algoritmo_genetico(grafica_bipartita_k_5, tours_totales_5, tam_poblacion_k_5, 20, 0.25)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es", mejor_solucion_geneticos_k_5_3[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_5_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_5_3),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es [0, 7, 2, 5, 3, 8, 1, 6, 4, 9] cuya distancia total es 40.73489889753606 unidades.
El tiempo del Algoritmo Genético fue de 0.12178683280944824 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es [0, 7, 3, 8, 1, 5, 2, 6, 4, 9] cuya distancia total es 44.76938232498737 unidades.
El tiempo del Algoritmo Genético fue de 0.14312052726745605 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es [0, 7, 2, 5, 3, 8, 1, 6, 4, 9] cuya distancia total es 40.734898897536056 unidades.
El tiempo del Algoritmo Genético fue de 0.15283846855163574 segundos  



# Sección de pruebas de las metaheurísticas para la gráfica bipartita completa $K_{6,6}$

## Código de la gráfica bipartita completa $K_{6,6}$

In [ ]:
# A continuación, se crea la matriz de distancias para la gráfica K_{6,6}.
tam_part_7 = 6
tam_part_8= 6
grafica_bipartita_k_6 = distancias_grafica_bipartita(tam_part_7,tam_part_8)

# Se define la lista de todos los posibles tours distintos en la gráfica $K_{6,6}$, con base en la matriz de distancias.
tours_totales_6=vecindad_proyecto([0,6,1,7,2,8,3,9,4,10,5,11])
tours_totales_6.append([0,6,1,7,2,8,3,9,4,10,5,11])

## Búsqueda Local y GRASP para la gráfica bipartita completa $K_{6,6}$

In [ ]:
# Búsqueda Local aplicada al problema.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_busqueda_local_k_6=time.time()
# Se proporciona una solución inicial al azar.
solucion_inicial_k_6 = random.choice(tours_totales_6)
# Se obtiene una mejor solución, usando Búsqueda Local para cada una de las instancias.
mejor_solucion_k_6 = Busqueda_local(solucion_inicial_k_6, grafica_bipartita_k_6, distancia_ciclo, vecindad_proyecto)
# Se imprime la mejor solución y su distancia total.
print("El ciclo hamiltoniano más corto con Búsqueda Local es",mejor_solucion_k_6,"cuya distancia total es",distancia_ciclo(grafica_bipartita_k_6, mejor_solucion_k_6),"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Local fue de %s segundos. " % (time.time() - t_inicial_busqueda_local_k_6),"\n")

# GRASP con construcción V aplicado al problema.

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.3.
t_inicial_grasp_v_k_6=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_6 = grasp_v(grafica_bipartita_k_6, tours_totales_6, distancia_ciclo, vecindad_proyecto, 10, 0.3)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es",mejor_solucion_grasp_v_k_6[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_6[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_6),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.5.
t_inicial_grasp_v_k_6_2=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_6_2 = grasp_v(grafica_bipartita_k_6, tours_totales_6, distancia_ciclo, vecindad_proyecto, 10, 0.5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es",mejor_solucion_grasp_v_k_6_2[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_6_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_6_2),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.7.
t_inicial_grasp_v_k_6_3=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_6_3 = grasp_v(grafica_bipartita_k_6, tours_totales_6, distancia_ciclo, vecindad_proyecto, 10, 0.7)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es",mejor_solucion_grasp_v_k_6_3[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_6_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_6_3),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.9.
t_inicial_grasp_v_k_6_4=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_6_4 = grasp_v(grafica_bipartita_k_6, tours_totales_6, distancia_ciclo, vecindad_proyecto, 10, 0.9)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.9 es",mejor_solucion_grasp_v_k_6_4[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_6_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_6_4),"\n")

El ciclo hamiltoniano más corto con Búsqueda Local es [0, 11, 1, 8, 2, 6, 3, 10, 4, 9, 5, 7] cuya distancia total es 43.49777234293608 unidades.
El tiempo de Búsqueda Local fue de 0.05435037612915039 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es [0, 11, 1, 10, 2, 6, 3, 8, 4, 9, 5, 7] cuya distancia total es 45.120477543759996 unidades.
El tiempo de GRASP con construcción V fue de 0.049306631088256836 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es [0, 10, 1, 6, 2, 8, 3, 11, 4, 9, 5, 7] cuya distancia total es 49.55952371318394 unidades.
El tiempo de GRASP con construcción V fue de 0.05492663383483887 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es [0, 11, 1, 10, 2, 8, 3, 7, 4, 9, 5, 6] cuya distancia total es 46.2435469918311 unidades.
El tiempo de GRASP con construcción V fue de 0.045110225677490234 segundos.  

El ciclo hamiltoniano más corto con GRASP con Constru

## Recocido Simulado para la gráfica bipartita completa $K_{6,6}$

In [ ]:
# Recocido Simulado aplicado al problema.

# Se establecen las temperaturas inicial y final
t0= 100
tf= 0.1
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_6=time.time()
# Se aplica el algoritmo al problema para alpha=0.6 y NREP=10.
mejor_solucion_recocido_k_6 = Recocido_simulado(solucion_inicial_k_6, grafica_bipartita_k_6, distancia_ciclo, t0, 0.6, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es",mejor_solucion_recocido_k_6[0],
      "cuya distancia total es",mejor_solucion_recocido_k_6[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_6),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_6_2=time.time()
# Se aplica el algoritmo al problema para alpha=0.7 y NREP=10.
mejor_solucion_recocido_k_6_2 = Recocido_simulado(solucion_inicial_k_6, grafica_bipartita_k_6, distancia_ciclo, t0, 0.7, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es",mejor_solucion_recocido_k_6_2[0],
      "cuya distancia total es",mejor_solucion_recocido_k_6_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_6_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_6_3=time.time()
# Se aplica el algoritmo al problema para alpha=0.8 y NREP=10.
mejor_solucion_recocido_k_6_3 = Recocido_simulado(solucion_inicial_k_6, grafica_bipartita_k_6, distancia_ciclo, t0, 0.8, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es",mejor_solucion_recocido_k_6_3[0],
      "cuya distancia total es",mejor_solucion_recocido_k_6_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_6_3),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_6_4=time.time()
# Se aplica el algoritmo al problema para alpha=0.9 y NREP=10.
mejor_solucion_recocido_k_6_4 = Recocido_simulado(solucion_inicial_k_6, grafica_bipartita_k_6, distancia_ciclo, t0, 0.9, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es",mejor_solucion_recocido_k_6_4[0],
      "cuya distancia total es",mejor_solucion_recocido_k_6_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_6_4),"\n")

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es [0, 11, 1, 10, 2, 8, 3, 7, 4, 9, 5, 6] cuya distancia total es 46.2435469918311 unidades.
El tiempo de Recocido Simulado fue de 0.054761648178100586 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es [0, 7, 1, 8, 2, 6, 3, 10, 4, 9, 5, 11] cuya distancia total es 44.53825833750871 unidades.
El tiempo de Recocido Simulado fue de 0.05354928970336914 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es [0, 11, 1, 6, 2, 8, 3, 10, 4, 9, 5, 7] cuya distancia total es 44.386917339666645 unidades.
El tiempo de Recocido Simulado fue de 0.09525418281555176 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es [0, 11, 1, 6, 2, 8, 3, 10, 4, 9, 5, 7] cuya distancia total es 44.386917339666645 unidades.
El tiempo de Recocido Simulado fue de 0.18825912475585938 segundos.  



## Búsqueda Tabú para la gráfica bipartita completa $K_{6,6}$

In [ ]:
# Búsqueda Tabú aplicada al problema.

# Se eligen aleatoriamente una cantidad de tours a prohibir.
tours_prohibidos_k_6=random.sample(tours_totales_6, 600)
# Se crea la lista tabú.
lista_tabu_k_6 = []
# Se agregan los ciclos hamiltonianos prohibidos a la lista.
lista_tabu_k_6.append(tours_prohibidos_k_6)
# Se elige aleatoriamente una solución inicial.
solucion_inicial_tabu_k_6 = random.choice(tours_totales_6)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_6=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_6 = Busqueda_tabu(solucion_inicial_tabu_k_6, grafica_bipartita_k_6, distancia_ciclo, lista_tabu_k_6, 10, 5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_6[0],
      "cuya distancia total es",mejor_solucion_tabu_k_6[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_6),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_6_2=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_6_2 = Busqueda_tabu(solucion_inicial_tabu_k_6, grafica_bipartita_k_6, distancia_ciclo, lista_tabu_k_6, 10, 10)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_6_2[0],
      "cuya distancia total es",mejor_solucion_tabu_k_6_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_6_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_6_3=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_6_3 = Busqueda_tabu(solucion_inicial_tabu_k_6, grafica_bipartita_k_6, distancia_ciclo, lista_tabu_k_6, 10, 20)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_6_3[0],
      "cuya distancia total es",mejor_solucion_tabu_k_6_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_6_3),"\n")

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 11, 1, 8, 2, 6, 3, 10, 4, 9, 5, 7] cuya distancia total es 43.49777234293608 unidades.
El tiempo de Búsqueda Tabú fue de 0.05471205711364746 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 11, 1, 6, 2, 8, 3, 10, 4, 9, 5, 7] cuya distancia total es 44.386917339666645 unidades.
El tiempo de Búsqueda Tabú fue de 0.05108380317687988 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 11, 1, 8, 2, 6, 3, 10, 4, 9, 5, 7] cuya distancia total es 43.49777234293608 unidades.
El tiempo de Búsqueda Tabú fue de 0.04947710037231445 segundos  



## Algoritmos Genéticos para la gráfica bipartita completa $K_{6,6}$

In [ ]:
# Algoritmo Genético aplicado al problema.


# Se asigna el tamaño de la población que sea menor o igual a la longitud de tours_totales
tam_poblacion_k_6 = len(tours_totales_6)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_6=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 15%.
mejor_solucion_geneticos_k_6 = algoritmo_genetico(grafica_bipartita_k_6, tours_totales_6, tam_poblacion_k_6, 20, 0.15)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es", mejor_solucion_geneticos_k_6[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_6[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_6),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_6_2=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 20%.
mejor_solucion_geneticos_k_6_2 = algoritmo_genetico(grafica_bipartita_k_6, tours_totales_6, tam_poblacion_k_6, 20, 0.2)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es", mejor_solucion_geneticos_k_6_2[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_6_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_6_2),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_6_3=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 25%.
mejor_solucion_geneticos_k_6_3 = algoritmo_genetico(grafica_bipartita_k_6, tours_totales_6, tam_poblacion_k_6, 20, 0.25)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es", mejor_solucion_geneticos_k_6_3[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_6_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_6_3),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es [0, 11, 3, 6, 4, 9, 5, 10, 1, 8, 2, 7] cuya distancia total es 40.16787272285287 unidades.
El tiempo del Algoritmo Genético fue de 1.8142540454864502 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es [0, 7, 5, 9, 4, 10, 1, 11, 3, 8, 2, 6] cuya distancia total es 40.62622870173418 unidades.
El tiempo del Algoritmo Genético fue de 2.8981375694274902 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es [0, 11, 3, 7, 1, 10, 5, 9, 4, 6, 2, 8] cuya distancia total es 40.90165741603239 unidades.
El tiempo del Algoritmo Genético fue de 1.6715221405029297 segundos  



# Sección de pruebas de las metaheurísticas para la gráfica bipartita completa $K_{7,7}$

## Código de la gráfica bipartita completa $K_{7,7}$

In [ ]:
# A continuación, se crea la matriz de distancias para la gráfica K_{7,7}.
tam_part_9 = 7
tam_part_10= 7
grafica_bipartita_k_7 = distancias_grafica_bipartita(tam_part_9,tam_part_10)

# Se define la lista de todos los posibles tours distintos en la gráfica $K_{7,7}$, con base en la matriz de distancias.
tours_totales_7=vecindad_proyecto([0,7,1,8,2,9,3,10,4,11,5,12,6,13])
tours_totales_7.append([0,7,1,8,2,9,3,10,4,11,5,12,6,13])

## Búsqueda Local y GRASP para la gráfica bipartita completa $K_{7,7}$

In [ ]:
# Búsqueda Local aplicada al problema.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_busqueda_local_k_7=time.time()
# Se proporciona una solución inicial al azar.
solucion_inicial_k_7 = random.choice(tours_totales_7)
# Se obtiene una mejor solución, usando Búsqueda Local para cada una de las instancias.
mejor_solucion_k_7 = Busqueda_local(solucion_inicial_k_7, grafica_bipartita_k_7, distancia_ciclo, vecindad_proyecto)
# Se imprime la mejor solución y su distancia total.
print("El ciclo hamiltoniano más corto con Búsqueda Local es",mejor_solucion_k_7,"cuya distancia total es",distancia_ciclo(grafica_bipartita_k_7, mejor_solucion_k_7),"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Local fue de %s segundos. " % (time.time() - t_inicial_busqueda_local_k_7),"\n")

# GRASP con construcción V aplicado al problema.

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.3.
t_inicial_grasp_v_k_7=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_7 = grasp_v(grafica_bipartita_k_7, tours_totales_7, distancia_ciclo, vecindad_proyecto, 10, 0.3)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es",mejor_solucion_grasp_v_k_7[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_7[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_7),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.5.
t_inicial_grasp_v_k_7_2=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_7_2 = grasp_v(grafica_bipartita_k_7, tours_totales_7, distancia_ciclo, vecindad_proyecto, 10, 0.5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es",mejor_solucion_grasp_v_k_7_2[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_7_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_7_2),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.7.
t_inicial_grasp_v_k_7_3=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_7_3 = grasp_v(grafica_bipartita_k_7, tours_totales_7, distancia_ciclo, vecindad_proyecto, 10, 0.7)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es",mejor_solucion_grasp_v_k_7_3[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_7_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_7_3),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.9.
t_inicial_grasp_v_k_7_4=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_7_4 = grasp_v(grafica_bipartita_k_7, tours_totales_7, distancia_ciclo, vecindad_proyecto, 10, 0.9)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.9 es",mejor_solucion_grasp_v_k_7_4[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_7_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_7_4),"\n")

El ciclo hamiltoniano más corto con Búsqueda Local es [0, 13, 1, 11, 2, 10, 3, 8, 4, 9, 5, 7, 6, 12] cuya distancia total es 58.184718617890994 unidades.
El tiempo de Búsqueda Local fue de 0.3665289878845215 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es [0, 12, 1, 9, 2, 10, 3, 11, 4, 7, 5, 8, 6, 13] cuya distancia total es 60.866373769070506 unidades.
El tiempo de GRASP con construcción V fue de 0.39051365852355957 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es [0, 13, 1, 9, 2, 10, 3, 11, 4, 8, 5, 7, 6, 12] cuya distancia total es 59.4624403917075 unidades.
El tiempo de GRASP con construcción V fue de 0.42852330207824707 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es [0, 13, 1, 7, 2, 8, 3, 11, 4, 12, 5, 10, 6, 9] cuya distancia total es 68.91076434838979 unidades.
El tiempo de GRASP con construcción V fue de 0.35439586639404297 segundos.  

El ciclo hamiltoniano m

## Recocido Simulado para la gráfica bipartita completa $K_{7,7}$

In [ ]:
# Recocido Simulado aplicado al problema.

# Se establecen las temperaturas inicial y final
t0= 100
tf= 0.1
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_7=time.time()
# Se aplica el algoritmo al problema para alpha=0.6 y NREP=10.
mejor_solucion_recocido_k_7 = Recocido_simulado(solucion_inicial_k_7, grafica_bipartita_k_7, distancia_ciclo, t0, 0.6, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es",mejor_solucion_recocido_k_7[0],
      "cuya distancia total es",mejor_solucion_recocido_k_7[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_7),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_7_2=time.time()
# Se aplica el algoritmo al problema para alpha=0.7 y NREP=10.
mejor_solucion_recocido_k_7_2 = Recocido_simulado(solucion_inicial_k_7, grafica_bipartita_k_7, distancia_ciclo, t0, 0.7, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es",mejor_solucion_recocido_k_7_2[0],
      "cuya distancia total es",mejor_solucion_recocido_k_7_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_7_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_7_3=time.time()
# Se aplica el algoritmo al problema para alpha=0.8 y NREP=10.
mejor_solucion_recocido_k_7_3 = Recocido_simulado(solucion_inicial_k_7, grafica_bipartita_k_7, distancia_ciclo, t0, 0.8, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es",mejor_solucion_recocido_k_7_3[0],
      "cuya distancia total es",mejor_solucion_recocido_k_7_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_7_3),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_7_4=time.time()
# Se aplica el algoritmo al problema para alpha=0.9 y NREP=10.
mejor_solucion_recocido_k_7_4 = Recocido_simulado(solucion_inicial_k_7, grafica_bipartita_k_7, distancia_ciclo, t0, 0.9, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es",mejor_solucion_recocido_k_7_4[0],
      "cuya distancia total es",mejor_solucion_recocido_k_7_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_7_4),"\n")

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es [0, 9, 1, 8, 2, 10, 3, 12, 4, 11, 5, 7, 6, 13] cuya distancia total es 62.905519564714524 unidades.
El tiempo de Recocido Simulado fue de 0.7333612442016602 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es [0, 13, 1, 11, 2, 10, 3, 7, 4, 8, 5, 9, 6, 12] cuya distancia total es 62.366997819999284 unidades.
El tiempo de Recocido Simulado fue de 1.3845605850219727 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es [0, 12, 1, 10, 2, 8, 3, 9, 4, 11, 5, 7, 6, 13] cuya distancia total es 62.229543985991434 unidades.
El tiempo de Recocido Simulado fue de 2.234727621078491 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es [0, 12, 1, 9, 2, 10, 3, 8, 4, 7, 5, 11, 6, 13] cuya distancia total es 59.62100075160353 unidades.
El tiempo de Recocido Simulado fue de 3.314847230911255 segundos.  



## Búsqueda Tabú para la gráfica bipartita completa $K_{7,7}$

In [ ]:
# Búsqueda Tabú aplicada al problema.

# Se eligen aleatoriamente una cantidad de tours a prohibir.
tours_prohibidos_k_7=random.sample(tours_totales_7, 200)
# Se crea la lista tabú.
lista_tabu_k_7 = []
# Se agregan los ciclos hamiltonianos prohibidos a la lista.
lista_tabu_k_7.append(tours_prohibidos_k_7)
# Se elige aleatoriamente una solución inicial.
solucion_inicial_tabu_k_7 = random.choice(tours_totales_7)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_7=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_7 = Busqueda_tabu(solucion_inicial_tabu_k_7, grafica_bipartita_k_7, distancia_ciclo, lista_tabu_k_7, 10, 5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_7[0],
      "cuya distancia total es",mejor_solucion_tabu_k_7[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_7),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_7_2=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_7_2 = Busqueda_tabu(solucion_inicial_tabu_k_7, grafica_bipartita_k_7, distancia_ciclo, lista_tabu_k_7, 10, 10)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_7_2[0],
      "cuya distancia total es",mejor_solucion_tabu_k_7_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_7_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_7_3=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_7_3 = Busqueda_tabu(solucion_inicial_tabu_k_7, grafica_bipartita_k_7, distancia_ciclo, lista_tabu_k_7, 10, 20)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_7_3[0],
      "cuya distancia total es",mejor_solucion_tabu_k_7_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_7_3),"\n")

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 13, 1, 11, 2, 10, 3, 8, 4, 9, 5, 7, 6, 12] cuya distancia total es 58.184718617890994 unidades.
El tiempo de Búsqueda Tabú fue de 0.4825780391693115 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 13, 1, 11, 2, 10, 3, 9, 4, 8, 5, 7, 6, 12] cuya distancia total es 58.20104381965375 unidades.
El tiempo de Búsqueda Tabú fue de 0.45330095291137695 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 13, 1, 11, 2, 10, 3, 8, 4, 9, 5, 7, 6, 12] cuya distancia total es 58.184718617890994 unidades.
El tiempo de Búsqueda Tabú fue de 0.46883320808410645 segundos  



## Algoritmos Genéticos para la gráfica bipartita completa $K_{7,7}$

In [ ]:
# Algoritmo Genético aplicado al problema.


# Se asigna el tamaño de la población que sea menor o igual a la longitud de tours_totales
tam_poblacion_k_7 = len(tours_totales_7)// 4
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_7=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 15%.
mejor_solucion_geneticos_k_7 = algoritmo_genetico(grafica_bipartita_k_7, tours_totales_7, tam_poblacion_k_7, 20, 0.15)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es", mejor_solucion_geneticos_k_7[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_7[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_7),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_7_2=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 20%.
mejor_solucion_geneticos_k_7_2 = algoritmo_genetico(grafica_bipartita_k_7, tours_totales_7, tam_poblacion_k_7, 20, 0.2)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es", mejor_solucion_geneticos_k_7_2[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_7_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_7_2),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_7_3=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 25%.
mejor_solucion_geneticos_k_7_3 = algoritmo_genetico(grafica_bipartita_k_7, tours_totales_7, tam_poblacion_k_7, 20, 0.25)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es", mejor_solucion_geneticos_k_7_3[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_7_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_7_3),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es [0, 13, 4, 12, 3, 8, 5, 10, 2, 7, 1, 11, 6, 9] cuya distancia total es 50.942023866023234 unidades.
El tiempo del Algoritmo Genético fue de 5.125860214233398 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es [0, 12, 1, 11, 6, 9, 4, 13, 3, 8, 5, 7, 2, 10] cuya distancia total es 54.39084637323834 unidades.
El tiempo del Algoritmo Genético fue de 6.318444013595581 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es [0, 12, 4, 10, 3, 8, 5, 9, 1, 11, 6, 7, 2, 13] cuya distancia total es 52.422110036275505 unidades.
El tiempo del Algoritmo Genético fue de 4.9705541133880615 segundos  



# Sección de pruebas de las metaheurísticas para la gráfica bipartita completa $K_{8,8}$

## Código de la gráfica bipartita completa $K_{8,8}$

In [ ]:
# A continuación, se crea la matriz de distancias para la gráfica K_{8,8}.
tam_part_11 = 8
tam_part_12= 8
grafica_bipartita_k_8 = distancias_grafica_bipartita(tam_part_11,tam_part_12)

# Se define la lista de todos los posibles tours distintos en la gráfica $K_{8,8}$, con base en la matriz de distancias.
tours_totales_8=vecindad_proyecto([0,8,1,9,2,10,3,11,4,12,5,13,6,14,7,15])
tours_totales_8.append([0,8,1,9,2,10,3,11,4,12,5,13,6,14,7,15])

## Búsqueda Local y GRASP para la gráfica bipartita completa $K_{8,8}$

In [ ]:
# Búsqueda Local aplicada al problema.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_busqueda_local_k_8=time.time()
# Se proporciona una solución inicial al azar.
solucion_inicial_k_8 = random.choice(tours_totales_8)
# Se obtiene una mejor solución, usando Búsqueda Local para cada una de las instancias.
mejor_solucion_k_8 = Busqueda_local(solucion_inicial_k_8, grafica_bipartita_k_8, distancia_ciclo, vecindad_proyecto)
# Se imprime la mejor solución y su distancia total.
print("El ciclo hamiltoniano más corto con Búsqueda Local es",mejor_solucion_k_8,"cuya distancia total es",distancia_ciclo(grafica_bipartita_k_8, mejor_solucion_k_8),"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Local fue de %s segundos. " % (time.time() - t_inicial_busqueda_local_k_8),"\n")

# GRASP con construcción V aplicado al problema.

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.3.
t_inicial_grasp_v_k_8=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_8 = grasp_v(grafica_bipartita_k_8, tours_totales_8, distancia_ciclo, vecindad_proyecto, 10, 0.3)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es",mejor_solucion_grasp_v_k_8[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_8[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_8),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.5.
t_inicial_grasp_v_k_8_2=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_8_2 = grasp_v(grafica_bipartita_k_8, tours_totales_8, distancia_ciclo, vecindad_proyecto, 10, 0.5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es",mejor_solucion_grasp_v_k_8_2[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_8_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_8_2),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.7.
t_inicial_grasp_v_k_8_3=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_8_3 = grasp_v(grafica_bipartita_k_8, tours_totales_8, distancia_ciclo, vecindad_proyecto, 10, 0.7)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es",mejor_solucion_grasp_v_k_8_3[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_8_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_8_3),"\n")

# Se calcula el tiempo inicial de la metaheurística para 10 iteraciones y alpha=0.9.
t_inicial_grasp_v_k_8_4=time.time()
# Se aplica el algoritmo al problema para la instancia seleccionada.
mejor_solucion_grasp_v_k_8_4 = grasp_v(grafica_bipartita_k_8, tours_totales_8, distancia_ciclo, vecindad_proyecto, 10, 0.9)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.9 es",mejor_solucion_grasp_v_k_8_4[0],
      "cuya distancia total es",mejor_solucion_grasp_v_k_8_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de GRASP con construcción V fue de %s segundos. " % (time.time() - t_inicial_grasp_v_k_8_4),"\n")

El ciclo hamiltoniano más corto con Búsqueda Local es [0, 13, 1, 9, 2, 15, 3, 11, 4, 8, 5, 12, 6, 10, 7, 14] cuya distancia total es 47.217838878689754 unidades.
El tiempo de Búsqueda Local fue de 5.073750019073486 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.3 es [0, 14, 1, 13, 2, 15, 3, 11, 4, 12, 5, 9, 6, 10, 7, 8] cuya distancia total es 57.04515054516882 unidades.
El tiempo de GRASP con construcción V fue de 3.455970287322998 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.5 es [0, 14, 1, 8, 2, 13, 3, 15, 4, 12, 5, 11, 6, 10, 7, 9] cuya distancia total es 66.81095958664882 unidades.
El tiempo de GRASP con construcción V fue de 4.682171821594238 segundos.  

El ciclo hamiltoniano más corto con GRASP con Construcción V y alfa=0.7 es [0, 9, 1, 8, 2, 15, 3, 10, 4, 11, 5, 13, 6, 14, 7, 12] cuya distancia total es 62.11038558150782 unidades.
El tiempo de GRASP con construcción V fue de 3.4238266944885254 segundos. 

## Recocido Simulado para la gráfica bipartita completa $K_{8,8}$

In [ ]:
# Recocido Simulado aplicado al problema.

# Se establecen las temperaturas inicial y final
t0= 100
tf= 0.1
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_8=time.time()
# Se aplica el algoritmo al problema para alpha=0.6 y NREP=10.
mejor_solucion_recocido_k_8 = Recocido_simulado(solucion_inicial_k_8, grafica_bipartita_k_8, distancia_ciclo, t0, 0.6, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es",mejor_solucion_recocido_k_8[0],
      "cuya distancia total es",mejor_solucion_recocido_k_8[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_8),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_8_2=time.time()
# Se aplica el algoritmo al problema para alpha=0.7 y NREP=10.
mejor_solucion_recocido_k_8_2 = Recocido_simulado(solucion_inicial_k_8, grafica_bipartita_k_8, distancia_ciclo, t0, 0.7, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es",mejor_solucion_recocido_k_8_2[0],
      "cuya distancia total es",mejor_solucion_recocido_k_8_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_8_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_8_3=time.time()
# Se aplica el algoritmo al problema para alpha=0.8 y NREP=10.
mejor_solucion_recocido_k_8_3 = Recocido_simulado(solucion_inicial_k_8, grafica_bipartita_k_8, distancia_ciclo, t0, 0.8, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es",mejor_solucion_recocido_k_8_3[0],
      "cuya distancia total es",mejor_solucion_recocido_k_8_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_8_3),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_recocido_k_8_4=time.time()
# Se aplica el algoritmo al problema para alpha=0.9 y NREP=10.
mejor_solucion_recocido_k_8_4 = Recocido_simulado(solucion_inicial_k_8, grafica_bipartita_k_8, distancia_ciclo, t0, 0.9, 10, tf)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es",mejor_solucion_recocido_k_8_4[0],
      "cuya distancia total es",mejor_solucion_recocido_k_8_4[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Recocido Simulado fue de %s segundos. " % (time.time() - t_inicial_recocido_k_8_4),"\n")

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.6 es [0, 9, 1, 8, 2, 11, 3, 12, 4, 15, 5, 14, 6, 10, 7, 13] cuya distancia total es 59.851933481452676 unidades.
El tiempo de Recocido Simulado fue de 11.971862316131592 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.7 es [0, 13, 1, 8, 2, 15, 3, 11, 4, 10, 5, 12, 6, 9, 7, 14] cuya distancia total es 52.99315864500235 unidades.
El tiempo de Recocido Simulado fue de 16.988980293273926 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.8 es [0, 14, 1, 9, 2, 11, 3, 15, 4, 8, 5, 12, 6, 10, 7, 13] cuya distancia total es 56.14002774039234 unidades.
El tiempo de Recocido Simulado fue de 28.008453607559204 segundos.  

El ciclo hamiltoniano más corto con Recocido Simulado con alfa=0.9 es [0, 12, 1, 9, 2, 15, 3, 10, 4, 11, 5, 8, 6, 13, 7, 14] cuya distancia total es 58.38137727383606 unidades.
El tiempo de Recocido Simulado fue de 55.32536244392395 segundos.  



## Búsqueda Tabú para la gráfica bipartita completa $K_{8,8}$

In [ ]:
# Búsqueda Tabú aplicada al problema.

# Se eligen aleatoriamente una cantidad de tours a prohibir.
tours_prohibidos_k_8=random.sample(tours_totales_8, 6000)
# Se crea la lista tabú.
lista_tabu_k_8 = []
# Se agregan los ciclos hamiltonianos prohibidos a la lista.
lista_tabu_k_8.append(tours_prohibidos_k_8)
# Se elige aleatoriamente una solución inicial.
solucion_inicial_tabu_k_8 = random.choice(tours_totales_8)
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_8=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_8 = Busqueda_tabu(solucion_inicial_tabu_k_8, grafica_bipartita_k_8, distancia_ciclo, lista_tabu_k_8, 10, 5)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_8[0],
      "cuya distancia total es",mejor_solucion_tabu_k_8[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_8),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_8_2=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_8_2 = Busqueda_tabu(solucion_inicial_tabu_k_8, grafica_bipartita_k_8, distancia_ciclo, lista_tabu_k_8, 10, 10)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_8_2[0],
      "cuya distancia total es",mejor_solucion_tabu_k_8_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_8_2),"\n")

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_tabu_k_8_3=time.time()
# Se aplica el algoritmo al problema.
mejor_solucion_tabu_k_8_3 = Busqueda_tabu(solucion_inicial_tabu_k_8, grafica_bipartita_k_8, distancia_ciclo, lista_tabu_k_8, 10, 20)
# Se imprime el resultado final.
print("El ciclo hamiltoniano más corto con Búsqueda Tabú es",mejor_solucion_tabu_k_8_3[0],
      "cuya distancia total es",mejor_solucion_tabu_k_8_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo de Búsqueda Tabú fue de %s segundos " % (time.time() - t_inicial_tabu_k_8_3),"\n")

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 13, 1, 9, 2, 15, 3, 11, 4, 8, 5, 12, 6, 10, 7, 14] cuya distancia total es 47.217838878689754 unidades.
El tiempo de Búsqueda Tabú fue de 3.762289047241211 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 13, 1, 9, 2, 15, 3, 11, 4, 12, 5, 8, 6, 10, 7, 14] cuya distancia total es 47.62848864136183 unidades.
El tiempo de Búsqueda Tabú fue de 4.967705011367798 segundos  

El ciclo hamiltoniano más corto con Búsqueda Tabú es [0, 13, 1, 9, 2, 15, 3, 11, 4, 8, 5, 12, 6, 10, 7, 14] cuya distancia total es 47.217838878689754 unidades.
El tiempo de Búsqueda Tabú fue de 3.7797622680664062 segundos  



## Algoritmos Genéticos para la gráfica bipartita completa $K_{8,8}$

In [ ]:
# Algoritmo Genético aplicado al problema.


# Se asigna el tamaño de la población que sea menor o igual a la longitud de tours_totales
tam_poblacion_k_8 = len(tours_totales_8) // 16
# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_8=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 15%.
mejor_solucion_geneticos_k_8 = algoritmo_genetico(grafica_bipartita_k_8, tours_totales_8, tam_poblacion_k_8, 20, 0.15)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es", mejor_solucion_geneticos_k_8[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_8[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_8),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_8_2=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 20%.
mejor_solucion_geneticos_k_8_2 = algoritmo_genetico(grafica_bipartita_k_8, tours_totales_8, tam_poblacion_k_8, 20, 0.2)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es", mejor_solucion_geneticos_k_8_2[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_8_2[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_8_2),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

# Se calcula el tiempo inicial de la metaheurística.
t_inicial_genetico_k_8_3=time.time()
# Se aplica el algoritmo al problema para 20 generaciones y con una tasa de mutación de 25%.
mejor_solucion_geneticos_k_8_3 = algoritmo_genetico(grafica_bipartita_k_8, tours_totales_8, tam_poblacion_k_8, 20, 0.25)
# Se imprime el resultado.
print("El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es", mejor_solucion_geneticos_k_8_3[0],
      "cuya distancia total es",mejor_solucion_geneticos_k_8_3[1],"unidades.")
# Se imprime el tiempo total del algoritmo.
print("El tiempo del Algoritmo Genético fue de %s segundos " % (time.time() - t_inicial_genetico_k_8_3),"\n")
# Observación. En algunos casos se llega al óptimo global. No obstante, si el valor de la función objetivo es el mismo al resto de heurísticas;
# aunque parezca una solución diferente, en realidad es la misma.

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 15% es [0, 14, 1, 9, 2, 15, 5, 11, 3, 10, 4, 12, 6, 13, 7, 8] cuya distancia total es 51.49126360828784 unidades.
El tiempo del Algoritmo Genético fue de 21.826042890548706 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 20% es [0, 8, 6, 14, 7, 10, 4, 12, 5, 11, 3, 15, 2, 9, 1, 13] cuya distancia total es 52.78663217048299 unidades.
El tiempo del Algoritmo Genético fue de 20.49213933944702 segundos  

El ciclo hamiltoniano más corto con Algoritmos Genéticos para 20 generaciones y con tasa de mutación del 25% es [0, 9, 7, 13, 5, 15, 2, 11, 3, 8, 4, 10, 6, 12, 1, 14] cuya distancia total es 49.61578594279667 unidades.
El tiempo del Algoritmo Genético fue de 21.906620979309082 segundos  

